# SQL TWO
## Análise de Dados com SQL
### Aula 4 - OVER, ROWS BETWEEN, PRECEDING, FOLLOWING, CURRENT ROW, LAG
Média Móvel, Soma móvel, LAG, Soma acumulada

### 1 - Diretório
#### Uma boa prática é inicialmente limpar o diretório, ou seja, excluir os arquivos (caso já tenham sido criados)

In [0]:
%fs rm -r dbfs:/user/hive/warehouse/faturamentos

res2: Boolean = true

### 2 - Tabela faturamentos
#### Vamos criar uma tabela com dados fictícios para nossos tratamentos neste notebook. Utilizaremos DROP e CREATE TABLE para isso.

In [0]:
-- 1 - Criação da base de dados
DROP TABLE IF EXISTS FATURAMENTOS;

CREATE TABLE FATURAMENTOS(
  data_fechamento DATE,
  faturamento_mm FLOAT
);

### 3 - DESCRIBE
#### Podemos verificar o tipo dos dados de cada coluna com o DESCRIBE.

In [0]:
-- 2 - Verificando o tipo dos dados
DESCRIBE FATURAMENTOS;

col_name,data_type,comment
data_fechamento,date,null
faturamento_mm,float,null


### 4 - Inserindo dados na tabela:
 - INSERT INTO
 - nome da tabela
    - (colunas da tabela)
 - VALUES
    - (dados de cada coluna separados por vírgula)
##### Obs.: valores de data formatados como "ano-mes-dia"

In [0]:
-- 3 - Inserindo dados na tabela
INSERT INTO 
FATURAMENTOS
VALUES
('2025-01-01', 8),
('2025-02-01', 10),
('2025-03-01', 6),
('2025-04-01', 9),
('2025-05-01', 5),
('2025-06-01', 4),
('2025-07-01', 7),
('2025-08-01', 11),
('2025-09-01', 9),
('2025-10-01', 12),
('2025-11-01', 11),
('2025-12-01', 10);

num_affected_rows,num_inserted_rows
12,12


### 5 - Visualizando a base de dados

In [0]:
-- 4 - Visualizando a base de dados
SELECT
  *
FROM FATURAMENTOS;

data_fechamento,faturamento_mm
2025-01-01,8.0
2025-02-01,10.0
2025-03-01,6.0
2025-04-01,9.0
2025-05-01,5.0
2025-06-01,4.0
2025-07-01,7.0
2025-08-01,11.0
2025-09-01,9.0
2025-10-01,12.0


### 6 - OVER, ROWS BETWEEN, PRECEDING, FOLLOWING
#### Podemos criar uma média movel de 3 períodos, onde o "valor da média" da coluna "média móvel" ficará na mesma linha do "2º valor" da coluna "faturamento_mm", salvo quando houver apenas 2 valores nesta coluna, quando ocorrerá o resultado em diagonal.

In [0]:
-- 5 - Criando uma média móvel de 3 períodos de tempo (anterior, atual e próximo)
SELECT
  data_fechamento,
  faturamento_mm,
  ROUND(AVG(faturamento_mm) 
    OVER(ORDER BY data_fechamento DESC ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING), 2) AS media_movel
FROM FATURAMENTOS
ORDER BY data_fechamento;

data_fechamento,faturamento_mm,media_movel
2025-01-01,8.0,9.0
2025-02-01,10.0,8.0
2025-03-01,6.0,8.33
2025-04-01,9.0,6.67
2025-05-01,5.0,6.0
2025-06-01,4.0,5.33
2025-07-01,7.0,7.33
2025-08-01,11.0,9.0
2025-09-01,9.0,10.67
2025-10-01,12.0,10.67


### 7 - OVER, ROWS BETWEEN, PRECEDING, CURRENT ROW
#### Criando uma soma móvel de 2 períodos, onde o resultado ocorrerá na diagonal.

In [0]:
-- 6 - Criando uma soma móvel de 2 períodos de tempo (anterior e atual)
SELECT
  data_fechamento,
  faturamento_mm,
  ROUND(SUM(faturamento_mm) 
    OVER(ORDER BY data_fechamento DESC ROWS BETWEEN 1 PRECEDING AND CURRENT ROW), 2) AS soma_movel
FROM FATURAMENTOS
ORDER BY data_fechamento;

data_fechamento,faturamento_mm,soma_movel
2025-01-01,8.0,18.0
2025-02-01,10.0,16.0
2025-03-01,6.0,15.0
2025-04-01,9.0,14.0
2025-05-01,5.0,9.0
2025-06-01,4.0,11.0
2025-07-01,7.0,18.0
2025-08-01,11.0,20.0
2025-09-01,9.0,21.0
2025-10-01,12.0,23.0


### 8 - LAG, OVER
#### Podemos utilizar o LAG para criar uma deslocamento numa nova coluna com os valores deslocados.

In [0]:
-- 7 - Criando uma coluna de deslocamento com LAG e OVER
SELECT
  data_fechamento,
  faturamento_mm,
  LAG(faturamento_mm, 1, 0) 
    OVER(ORDER BY data_fechamento ASC) AS coluna_faturamento_deslocada
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,coluna_faturamento_deslocada
2025-01-01,8.0,0.0
2025-02-01,10.0,8.0
2025-03-01,6.0,10.0
2025-04-01,9.0,6.0
2025-05-01,5.0,9.0
2025-06-01,4.0,5.0
2025-07-01,7.0,4.0
2025-08-01,11.0,7.0
2025-09-01,9.0,11.0
2025-10-01,12.0,9.0


### 9 - LAG, OVER (MOM)
#### Uma vez que temos a coluna com valores deslocados, podemos criar uma nova coluna com a diferença das anteriores, ou seja, uma coluna de varição MOM (month over month).

In [0]:
-- 8 - Criando coluna MOM com a variação das colunas de faturamento
SELECT
  data_fechamento,
  faturamento_mm,
  LAG(faturamento_mm, 1, 0) 
    OVER(ORDER BY data_fechamento ASC) AS coluna_faturamento_deslocada,
  faturamento_mm / coluna_faturamento_deslocada - 1 AS coluna_variacao_mensal
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,coluna_faturamento_deslocada,coluna_variacao_mensal
2025-01-01,8.0,0.0,null
2025-02-01,10.0,8.0,0.25
2025-03-01,6.0,10.0,-0.4
2025-04-01,9.0,6.0,0.5
2025-05-01,5.0,9.0,-0.4444444444444444
2025-06-01,4.0,5.0,-0.19999999999999996
2025-07-01,7.0,4.0,0.75
2025-08-01,11.0,7.0,0.5714285714285714
2025-09-01,9.0,11.0,-0.18181818181818177
2025-10-01,12.0,9.0,0.33333333333333326


### 10 - LAG, OVER, COALESCE
#### Contudo, precisamos corrigir o valor NULL gerado na coluna MOM. Faremos isso com o COALESCE, que preencherá o valor NULL com 0.

In [0]:
-- 9 - Criando coluna MOM com a variação das colunas de faturamento, preenchendo valor nulo com COALESCE
SELECT
  data_fechamento,
  faturamento_mm,
  LAG(faturamento_mm, 1, 0) 
    OVER(ORDER BY data_fechamento ASC) AS coluna_faturamento_deslocada,
  COALESCE(faturamento_mm / coluna_faturamento_deslocada - 1, 0) AS variacao_mensal
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,coluna_faturamento_deslocada,variacao_mensal
2025-01-01,8.0,0.0,0.0
2025-02-01,10.0,8.0,0.25
2025-03-01,6.0,10.0,-0.4
2025-04-01,9.0,6.0,0.5
2025-05-01,5.0,9.0,-0.4444444444444444
2025-06-01,4.0,5.0,-0.19999999999999996
2025-07-01,7.0,4.0,0.75
2025-08-01,11.0,7.0,0.5714285714285714
2025-09-01,9.0,11.0,-0.18181818181818177
2025-10-01,12.0,9.0,0.33333333333333326


### 11 - LAG, OVER, COALESCE, ROUND
#### Após o tratamento da coluna MOM, podemos ajustar as casas decimais com o ROUND.

In [0]:
-- 10 - Criando coluna MOM com a variação das colunas de faturamento, preenchendo valor nulo com COALESCE
SELECT
  data_fechamento,
  faturamento_mm,
  LAG(faturamento_mm, 1, 0) 
    OVER(ORDER BY data_fechamento ASC) AS coluna_faturamento_deslocada,
  ROUND(COALESCE(faturamento_mm / coluna_faturamento_deslocada - 1, 0), 2) AS variacao_mensal
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,coluna_faturamento_deslocada,variacao_mensal
2025-01-01,8.0,0.0,0.0
2025-02-01,10.0,8.0,0.25
2025-03-01,6.0,10.0,-0.4
2025-04-01,9.0,6.0,0.5
2025-05-01,5.0,9.0,-0.44
2025-06-01,4.0,5.0,-0.2
2025-07-01,7.0,4.0,0.75
2025-08-01,11.0,7.0,0.57
2025-09-01,9.0,11.0,-0.18
2025-10-01,12.0,9.0,0.33


### 12 - LAG, OVER, COALESCE, ROUND
#### Podemos ainda melhorar nossa coluna de faturamento deslocado, inserindo o valor de faturamento anterior, ao invés de 0.

In [0]:
-- 11 - Criando coluna MOM com a variação das colunas de faturamento, preenchendo valor nulo com COALESCE, e o valor de faturamento na coluna deslocada com LAG
SELECT
  data_fechamento,
  faturamento_mm,
  LAG(faturamento_mm, 1, faturamento_mm) 
    OVER(ORDER BY data_fechamento ASC) AS coluna_faturamento_deslocada,
  ROUND(faturamento_mm / coluna_faturamento_deslocada - 1, 2) AS variacao_mensal
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,coluna_faturamento_deslocada,variacao_mensal
2025-01-01,8.0,8.0,0.0
2025-02-01,10.0,8.0,0.25
2025-03-01,6.0,10.0,-0.4
2025-04-01,9.0,6.0,0.5
2025-05-01,5.0,9.0,-0.44
2025-06-01,4.0,5.0,-0.2
2025-07-01,7.0,4.0,0.75
2025-08-01,11.0,7.0,0.57
2025-09-01,9.0,11.0,-0.18
2025-10-01,12.0,9.0,0.33


### 13 - SUM, OVER
#### Podemos criar uma coluna com a soma mensal acumulada com SUM e OVER.

In [0]:
-- 12 - Criando uma soma acumulada com SUM e OVER
SELECT
  data_fechamento,
  faturamento_mm,
  SUM(faturamento_mm) 
    OVER(ORDER BY data_fechamento ASC) AS soma_mensal_acumulada
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,soma_mensal_acumulada
2025-01-01,8.0,8.0
2025-02-01,10.0,18.0
2025-03-01,6.0,24.0
2025-04-01,9.0,33.0
2025-05-01,5.0,38.0
2025-06-01,4.0,42.0
2025-07-01,7.0,49.0
2025-08-01,11.0,60.0
2025-09-01,9.0,69.0
2025-10-01,12.0,81.0


### 14 - SUM, OVER com ROWS BETWEEN, UNBOUNDED PRECEDING, CURRENT ROW
#### Na verdade, por detrás da lógica anterior, estão as funções SUM, OVER com ROWS BETWEEN, UNBOUNDED PRECEDING, CURRENT ROW.

In [0]:
-- 13 - Criando uma soma acumulada com SUM, OVER, ROWS BETWEEN, UNBOUNDED PRECEDING e CURRENT ROW
SELECT
  data_fechamento,
  faturamento_mm,
  SUM(faturamento_mm) 
    OVER(ORDER BY data_fechamento ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS soma_acumulada
FROM FATURAMENTOS
ORDER BY data_fechamento ASC;

data_fechamento,faturamento_mm,soma_acumulada
2025-01-01,8.0,8.0
2025-02-01,10.0,18.0
2025-03-01,6.0,24.0
2025-04-01,9.0,33.0
2025-05-01,5.0,38.0
2025-06-01,4.0,42.0
2025-07-01,7.0,49.0
2025-08-01,11.0,60.0
2025-09-01,9.0,69.0
2025-10-01,12.0,81.0
